# PCP case study
## No chl prime

### Notebook helper function for pretty output

In [ ]:
from IPython.display import Markdown
from IPython.display import display


def print_md(markdown_printable):
    """Pretty render Markdown."""
    display(Markdown(str(markdown_printable)))


def print_yaml_file(file_path):
    """Pretty render yaml file."""
    with open(file_path) as f:
        print_md(f"```yaml\n{f.read()}\n```")

### Plotting functions ([`pyglotaran_extras`](https://github.com/s-weigand/pyglotaran-extras/commit/20da3593105fb839f86e668dc12dc9ca87c3b9ce) + `matplotlib`)

In [ ]:
import matplotlib.pyplot as plt
from pyglotaran_extras.plotting.plot_overview import plot_overview
from pyglotaran_extras.plotting.style import PlotStyle

plot_style = PlotStyle()
plt.rc("axes", prop_cycle=plot_style.cycler)
plt.rcParams["figure.figsize"] = (21, 14)

### Analysis functions

In [ ]:
from glotaran.analysis.optimize import optimize
from glotaran.io import load_dataset
from glotaran.io import load_model
from glotaran.io import load_parameters
from glotaran.project.scheme import Scheme

### Read data

In [ ]:
dataset = load_dataset("PCP490.ascii")
dataset

### Load model and parameters

In [ ]:
PCP_model = load_model("models/PCP-model.yml")
PCP_parameters = load_parameters("models/PCP-parameters.yml")
print_md(PCP_model.validate(parameters=PCP_parameters))

In [ ]:
print_md(PCP_model)

### Create scheme and optimize it

In [ ]:
PCP_scheme = Scheme(PCP_model, PCP_parameters, {"dataset1": dataset})
PCP_result = optimize(PCP_scheme)

In [ ]:
PCP_result.data["dataset1"]

### Result plots

In [ ]:
try:
    fig = plot_overview(PCP_result.data["dataset1"], linlog=False)
except ValueError:
    # residual data aren't sorted and the plot fails
    pass

In [ ]:
print_md(PCP_result.optimized_parameters)

## chl prime

### Load model and parameters

In [ ]:
PCP_chl_prime_model = load_model("models/PCP-chl-prime-model.yml")
PCP_chl_prime_parameters = load_parameters("models/PCP-chl-prime-parameters.yml")
print_md(PCP_model.validate(parameters=PCP_chl_prime_parameters))

In [ ]:
print_md(PCP_chl_prime_model)

### Create scheme and optimize it

In [ ]:
PCP_chl_prime_scheme = Scheme(PCP_chl_prime_model, PCP_chl_prime_parameters, {"dataset1": dataset})
PCP_chl_prime_result = optimize(PCP_chl_prime_scheme)

In [ ]:
PCP_chl_prime_result.data["dataset1"]

### Result plots

In [ ]:
try:
    fig = plot_overview(PCP_chl_prime_result.data["dataset1"], linlog=False)
except ValueError:
    # residual data aren't sorted and the plot fails
    pass

In [ ]:
print_md(PCP_chl_prime_result.optimized_parameters)